<a href="https://colab.research.google.com/github/rahulprasanth487/S3-STORAGE-GCS-migration/blob/main/HPOS_migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install google-auth google-auth-oauthlib google-cloud-storage

In [4]:
from google.cloud import storage
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os

LIST THE FILES

In [5]:
def list_bucket_contents_with_token(bucket_name, access_token):
    credentials = Credentials(token=access_token)

    client = storage.Client(credentials=credentials, project='qwiklabs-gcp-00-1795097857bb')

    bucket = client.bucket(bucket_name)

    blobs = bucket.list_blobs()

    for blob in blobs:
        print(blob.name)

In [6]:
b_name = "testing_migration_123"
acc_token="xxx"

In [7]:
list_bucket_contents_with_token(b_name, acc_token)

WhatsApp Image 2025-03-24 at 17.35.55_6adb7a11.jpg


LIST FILES FROM HPOS

In [8]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.6 MB/s eta 0:00:00


In [10]:
import boto3

def list_s3_files(bucket_name, prefix=''):

    s3 = boto3.client(
        's3',
        aws_access_key_id='xxx',
        aws_secret_access_key='xxx',
        region_name='us-east-1'
    )

    # Initialize variables
    files = []
    continuation_token = None

    while True:
        params = {
            'Bucket': bucket_name,
            'Prefix': prefix
        }

        if continuation_token:
            params['ContinuationToken'] = continuation_token

        response = s3.list_objects_v2(**params)

        if 'Contents' in response:
            for obj in response['Contents']:
                files.append(obj['Key'])

        if not response.get('IsTruncated'):
            break

        continuation_token = response.get('NextContinuationToken')

    return files

if __name__ == "__main__":
    bucket_name = "hpos-sample-buc"

    files = list_s3_files(bucket_name)
    for file in files:
        print(file)

data_A.png
data_H.png
data_P.png
data_R.png
data_V.png


UPLOAD FILES TO GCS bucket

In [19]:
import boto3
from google.oauth2.credentials import Credentials
from google.cloud import storage
import tempfile
import os

def migrate_s3_to_gcs(s3_bucket_name, gcs_bucket_name, access_token,
                      s3_prefix='', gcs_prefix='', project_id='qwiklabs-gcp-00-1795097857bb'):
    """
    Migrate files from an AWS S3 bucket to a Google Cloud Storage bucket.

    Args:
        s3_bucket_name (str): Source S3 bucket name
        gcs_bucket_name (str): Destination GCS bucket name
        access_token (str): Google Cloud access token
        s3_prefix (str): Optional prefix/folder in S3 to filter objects
        gcs_prefix (str): Optional prefix/folder in GCS to store objects
        project_id (str): GCP project ID
    """
    # Initialize S3 client
    s3 = boto3.client(
        's3',
        aws_access_key_id='xxxx',
        aws_secret_access_key='xxxx',
        region_name='us-east-1'
    )

    # Initialize GCS client
    credentials = Credentials(token=access_token)
    gcs_client = storage.Client(credentials=credentials, project=project_id)
    gcs_bucket = gcs_client.bucket(gcs_bucket_name)

    # List objects in S3 bucket
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=s3_bucket_name, Prefix=s3_prefix)

    # Process each object
    for page in pages:
        if 'Contents' not in page:
            print(f"No objects found in S3 bucket '{s3_bucket_name}' with prefix '{s3_prefix}'")
            continue

        for obj in page['Contents']:
            s3_key = obj['Key']

            # Skip folders/directories (objects that end with '/')
            if s3_key.endswith('/'):
                print(f"Skipping directory: {s3_key}")
                continue
            # Skip files inside directories by checking if there's a '/' in the key
            if '/' in s3_key:
                print(f"Skipping file in subdirectory: {s3_key}")
                continue

            print(f"Processing file: {s3_key}")

            # Create a temp file to store the S3 object content
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file_path = temp_file.name

            try:
                # Download from S3 to temp file
                s3.download_file(s3_bucket_name, s3_key, temp_file_path)

                # Determine GCS destination path
                file_name = os.path.basename(s3_key)
                if gcs_prefix:
                    gcs_destination = f"{gcs_prefix.rstrip('/')}/{file_name}"
                else:
                    gcs_destination = file_name

                # Upload to GCS
                blob = gcs_bucket.blob(gcs_destination)
                blob.upload_from_filename(temp_file_path)

                print(f"Migrated '{s3_key}' to '{gcs_destination}' in GCS bucket '{gcs_bucket_name}'")

            finally:
                # Clean up temp file
                if os.path.exists(temp_file_path):
                    os.remove(temp_file_path)

# Example usage
if __name__ == "__main__":
    # S3 configuration
    s3_bucket_name = "hpos-sample-buc"
    s3_prefix = ""  # Optional: leave as empty string for entire bucket

    # GCS configuration
    gcs_bucket_name = "testing_migration_123"
    gcs_prefix = "migrated-from-s3/"
    acc_token="xxxx"
    project_id = "qwiklabs-gcp-00-1795097857bb"

    # Run migration
    migrate_s3_to_gcs(s3_bucket_name, gcs_bucket_name, acc_token, s3_prefix, gcs_prefix, project_id)

Skipping directory: Images/
Skipping file in subdirectory: Images/Screenshot 2023-10-17 010409.png
Skipping file in subdirectory: Images/Screenshot 2023-10-17 010438.png
Skipping file in subdirectory: Images/Screenshot 2023-10-17 010519.png
Processing file: data_A.png
Migrated 'data_A.png' to 'migrated-from-s3/data_A.png' in GCS bucket 'testing_migration_123'
Processing file: data_H.png
Migrated 'data_H.png' to 'migrated-from-s3/data_H.png' in GCS bucket 'testing_migration_123'
Processing file: data_P.png
Migrated 'data_P.png' to 'migrated-from-s3/data_P.png' in GCS bucket 'testing_migration_123'
Processing file: data_R.png
Migrated 'data_R.png' to 'migrated-from-s3/data_R.png' in GCS bucket 'testing_migration_123'
Processing file: data_V.png
Migrated 'data_V.png' to 'migrated-from-s3/data_V.png' in GCS bucket 'testing_migration_123'


# MIGRATE DIRS

In [14]:
def migrate_s3_to_gcs(s3_bucket_name, gcs_bucket_name, access_token,
                      s3_prefix='', gcs_base_prefix='', project_id='qwiklabs-gcp-00-1795097857bb'):

    # Initialize S3 client
    s3 = boto3.client(
        's3',
        aws_access_key_id='xxxx',
        aws_secret_access_key='xxxx',
        region_name='us-east-1'
    )

    # Initialize GCS client
    credentials = Credentials(token=access_token)
    gcs_client = storage.Client(credentials=credentials, project=project_id)
    gcs_bucket = gcs_client.bucket(gcs_bucket_name)

    # List objects in S3 bucket
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=s3_bucket_name, Prefix=s3_prefix)

    # Track directories we've seen to create them in GCS
    processed_dirs = set()

    # Process each object
    for page in pages:
        if 'Contents' not in page:
            print(f"No objects found in S3 bucket '{s3_bucket_name}' with prefix '{s3_prefix}'")
            continue

        for obj in page['Contents']:
            s3_key = obj['Key']

            # Skip folder placeholder objects (objects that end with '/')
            if s3_key.endswith('/'):
                print(f"Skipping directory placeholder: {s3_key}")
                continue

            print(f"Processing file: {s3_key}")

            # Create a temp file to store the S3 object content
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file_path = temp_file.name

            try:
                # Download from S3 to temp file
                s3.download_file(s3_bucket_name, s3_key, temp_file_path)

                # Preserve directory structure in GCS
                if gcs_base_prefix:
                    gcs_destination = f"{gcs_base_prefix.rstrip('/')}/{s3_key}"
                else:
                    gcs_destination = s3_key

                # Create "directory" structure in GCS (not strictly necessary as GCS uses flat namespace,
                # but helps with organization and improves console browsing experience)
                dir_path = os.path.dirname(gcs_destination)
                if dir_path and dir_path not in processed_dirs:
                    # Create an empty directory marker object if needed
                    if dir_path:
                        dir_blob = gcs_bucket.blob(f"{dir_path}/")
                        dir_blob.upload_from_string('')
                        processed_dirs.add(dir_path)

                # Upload to GCS
                blob = gcs_bucket.blob(gcs_destination)
                blob.upload_from_filename(temp_file_path)

                print(f"Migrated '{s3_key}' to '{gcs_destination}' in GCS bucket '{gcs_bucket_name}'")

            finally:
                # Clean up temp file
                if os.path.exists(temp_file_path):
                    os.remove(temp_file_path)


In [15]:
def list_s3_directories(s3_bucket_name, prefix=''):

    s3 = boto3.client(
        's3',
        aws_access_key_id='xxxx',
        aws_secret_access_key='xxxx',
        region_name='us-east-1'
    )

    # Use the list_objects_v2 with delimiter to get directories
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=s3_bucket_name, Prefix=prefix, Delimiter='/')

    directories = []

    for page in pages:
        # Add common prefixes (directories)
        if 'CommonPrefixes' in page:
            for common_prefix in page['CommonPrefixes']:
                directories.append(common_prefix['Prefix'])

    return directories


In [16]:
def migrate_s3_directories_to_gcs(s3_bucket_name, gcs_bucket_name, access_token, project_id='qwiklabs-gcp-01-61d132ccd3b0'):

    # First, list top-level directories
    top_dirs = list_s3_directories(s3_bucket_name)

    # If no directories, migrate the root
    if not top_dirs:
        print("No directories found. Migrating files from root...")
        migrate_s3_to_gcs(s3_bucket_name, gcs_bucket_name, access_token,
                         s3_prefix='', gcs_base_prefix='', project_id=project_id)
        return

    # Process each directory
    for directory in top_dirs:
        print(f"Processing directory: {directory}")

        # Migrate this directory
        migrate_s3_to_gcs(
            s3_bucket_name=s3_bucket_name,
            gcs_bucket_name=gcs_bucket_name,
            access_token=access_token,
            s3_prefix=directory,
            gcs_base_prefix='',  # Keep the original structure
            project_id=project_id
        )

        # Recursively look for subdirectories and migrate them
        subdirs = list_s3_directories(s3_bucket_name, directory)
        for subdir in subdirs:
            print(f"Processing subdirectory: {subdir}")
            migrate_s3_to_gcs(
                s3_bucket_name=s3_bucket_name,
                gcs_bucket_name=gcs_bucket_name,
                access_token=access_token,
                s3_prefix=subdir,
                gcs_base_prefix='',  # Keep the original structure
                project_id=project_id
            )

#

In [17]:
if __name__ == "__main__":
    # S3 configuration
    s3_bucket_name = "hpos-sample-buc"

    # GCS configuration
    gcs_bucket_name = "testing_migration_123"
    access_token = "xxxxx"
    project_id = "qwiklabs-gcp-00-1795097857bb"

    # Option 1: Run migration on entire bucket, preserving directory structure
    migrate_s3_directories_to_gcs(s3_bucket_name, gcs_bucket_name, access_token, project_id)


Processing directory: Images/
Skipping directory placeholder: Images/
Processing file: Images/Screenshot 2023-10-17 010409.png
Migrated 'Images/Screenshot 2023-10-17 010409.png' to 'Images/Screenshot 2023-10-17 010409.png' in GCS bucket 'testing_migration_123'
Processing file: Images/Screenshot 2023-10-17 010438.png
Migrated 'Images/Screenshot 2023-10-17 010438.png' to 'Images/Screenshot 2023-10-17 010438.png' in GCS bucket 'testing_migration_123'
Processing file: Images/Screenshot 2023-10-17 010519.png
Migrated 'Images/Screenshot 2023-10-17 010519.png' to 'Images/Screenshot 2023-10-17 010519.png' in GCS bucket 'testing_migration_123'
